# 라이브러리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import tensorflow as tf
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, auc

# combine 데이터

## 데이터 전처리

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/서울시 빅데이터(2022-2)/논문/3. 폐업 여부 분류 모델/0) 데이터/data_combined.csv')

In [ ]:
data['폐업여부']=data['폐업여부'].apply(lambda x: 1 if x=="Y" else 0)
data['프랜차이즈여부']=data['프랜차이즈여부'].apply(lambda x: 1 if x=="Y" else 0)

In [ ]:
data['표준산업분류코드']=data['표준산업분류코드'].astype('category')
data['행정동코드']=data['행정동코드'].astype('category')
data['분기']=data['분기'].astype('category')

In [ ]:
data_encoded = pd.get_dummies(data, columns=['표준산업분류코드', '행정동코드', '분기'])
data_encoded = data_encoded.drop(['점포명'], axis=1)

In [ ]:
train_data = data_encoded[data_encoded['년도'] == 2021]
test_data = data_encoded[data_encoded['년도'] == 2022]

In [ ]:
train_data = train_data.drop(['년도'], axis=1)
test_data = test_data.drop(['년도'], axis=1)

In [ ]:
x_train = train_data.drop(['폐업여부'], axis=1)
y_train = train_data['폐업여부']
x_test = test_data.drop(['폐업여부'], axis=1)
y_test = test_data['폐업여부']

In [ ]:
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

## 모델링

### 기본

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
614/614 [==============================] - 3s 3ms/step - loss: 0.1804 - accuracy: 0.9567 - val_loss: 0.0994 - val_accuracy: 0.9798
Epoch 2/100
614/614 [==============================] - 2s 2ms/step - loss: 0.1630 - accuracy: 0.9599 - val_loss: 0.1016 - val_accuracy: 0.9798
Epoch 3/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1615 - accuracy: 0.9599 - val_loss: 0.1019 - val_accuracy: 0.9798
Epoch 4/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1601 - accuracy: 0.9599 - val_loss: 0.1033 - val_accuracy: 0.9798
Epoch 5/100
614/614 [==============================] - 2s 4ms/step - loss: 0.1580 - accuracy: 0.9599 - val_loss: 0.1021 - val_accuracy: 0.9798
Epoch 6/100
614/614 [==============================] - 3s 4ms/step - loss: 0.1560 - accuracy: 0.9599 - val_loss: 0.1049 - val_accuracy: 0.9798
Epoch 7/100
614/614 [==============================] - 2s 4ms/step - loss: 0.1521 - accuracy: 0.9599 - val_loss: 0.1068 - val_accuracy: 0.9798

In [ ]:
y_pred = model.predict(x_test)

roc_score = roc_auc_score(y_test, y_pred)
print('ROC AUC 값: {0:.4f}'.format(roc_score))

precision, recall, _ = precision_recall_curve(y_test, y_pred)
pr_auc_score = auc(recall, precision)
print("PR AUC:", pr_auc_score)

725/725 [==============================] - 1s 1ms/step
ROC AUC 값: 0.5379
PR AUC: 0.04166613702070662


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
614/614 [==============================] - 3s 3ms/step - loss: 0.1749 - accuracy: 0.9599 - val_loss: 0.0996 - val_accuracy: 0.9798
Epoch 2/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1636 - accuracy: 0.9599 - val_loss: 0.1015 - val_accuracy: 0.9798
Epoch 3/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1608 - accuracy: 0.9599 - val_loss: 0.1013 - val_accuracy: 0.9798
Epoch 4/100
614/614 [==============================] - 2s 4ms/step - loss: 0.1591 - accuracy: 0.9599 - val_loss: 0.1053 - val_accuracy: 0.9798
Epoch 5/100
614/614 [==============================] - 3s 5ms/step - loss: 0.1563 - accuracy: 0.9599 - val_loss: 0.1049 - val_accuracy: 0.9798
Epoch 6/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1531 - accuracy: 0.9599 - val_loss: 0.1091 - val_accuracy: 0.9798
Epoch 7/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1480 - accuracy: 0.9599 - val_loss: 0.1088 - val_accuracy: 0.9798

In [ ]:
y_pred = model.predict(x_test)

roc_score = roc_auc_score(y_test, y_pred)
print('ROC AUC 값: {0:.4f}'.format(roc_score))

precision, recall, _ = precision_recall_curve(y_test, y_pred)
pr_auc_score = auc(recall, precision)
print("PR AUC:", pr_auc_score)

725/725 [==============================] - 1s 1ms/step
ROC AUC 값: 0.5357
PR AUC: 0.04127791103458622


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
614/614 [==============================] - 3s 3ms/step - loss: 0.1808 - accuracy: 0.9597 - val_loss: 0.0989 - val_accuracy: 0.9798
Epoch 2/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1639 - accuracy: 0.9599 - val_loss: 0.0990 - val_accuracy: 0.9798
Epoch 3/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1616 - accuracy: 0.9599 - val_loss: 0.1016 - val_accuracy: 0.9798
Epoch 4/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1595 - accuracy: 0.9599 - val_loss: 0.1028 - val_accuracy: 0.9798
Epoch 5/100
614/614 [==============================] - 3s 5ms/step - loss: 0.1569 - accuracy: 0.9599 - val_loss: 0.1057 - val_accuracy: 0.9798
Epoch 6/100
614/614 [==============================] - 2s 4ms/step - loss: 0.1535 - accuracy: 0.9599 - val_loss: 0.1096 - val_accuracy: 0.9798
Epoch 7/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1491 - accuracy: 0.9599 - val_loss: 0.1172 - val_accuracy: 0.9798

In [ ]:
y_pred = model.predict(x_test)

roc_score = roc_auc_score(y_test, y_pred)
print('ROC AUC 값: {0:.4f}'.format(roc_score))

precision, recall, _ = precision_recall_curve(y_test, y_pred)
pr_auc_score = auc(recall, precision)
print("PR AUC:", pr_auc_score)

725/725 [==============================] - 1s 1ms/step
ROC AUC 값: 0.5373
PR AUC: 0.04181837735594442


### 앙상블

In [ ]:
roc_auc_scores = []
pr_auc_scores = []
final_y_train_pred_proba = []
final_y_test_pred_proba = []

num_iterations = 20

for iteration in range(num_iterations):

    non_closed_data = train_data[train_data['폐업여부'] == 0].sample(n=5000, random_state= iteration+426)
    closed_data = train_data[train_data['폐업여부'] == 1]

    new_train_data = pd.concat([non_closed_data, closed_data], axis=0)

    new_x_train = new_train_data.drop(['폐업여부'], axis=1)
    new_y_train = new_train_data['폐업여부']

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    new_x_train = tf.convert_to_tensor(new_x_train, dtype=tf.float32)
    new_y_train = tf.convert_to_tensor(new_y_train, dtype=tf.float32)

    model.fit(new_x_train, new_y_train, epochs=50, batch_size=64, validation_split=0.2)

    final_y_train_pred_proba.append(model.predict(x_train))
    final_y_test_pred_proba.append(model.predict(x_test))

final_y_train_pred_proba = np.mean(final_y_train_pred_proba, axis=0)
final_y_test_pred_proba = np.mean(final_y_test_pred_proba, axis=0)

final_roc_auc_test = roc_auc_score(y_train,final_y_train_pred_proba)
print("Final Train ROC AUC:", final_roc_auc_test)
precision, recall, _ = precision_recall_curve(y_train,final_y_train_pred_proba)
final_pr_auc_test = auc(recall, precision)
print("Final Train PR AUC:", final_pr_auc_test)

final_roc_auc_test = roc_auc_score(y_test, final_y_test_pred_proba)
print("Final Test ROC AUC:", final_roc_auc_test)
precision, recall, _ = precision_recall_curve(y_test, final_y_test_pred_proba)
final_pr_auc_test = auc(recall, precision)
print("Final Test PR AUC:", final_pr_auc_test)

Epoch 1/50
85/85 [==============================] - 1s 5ms/step - loss: 0.2864 - accuracy: 0.9230 - val_loss: 3.7848 - val_accuracy: 0.0000e+00
Epoch 2/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1653 - accuracy: 0.9230 - val_loss: 6.3255 - val_accuracy: 0.0000e+00
Epoch 3/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1448 - accuracy: 0.9230 - val_loss: 9.3395 - val_accuracy: 0.0000e+00
Epoch 4/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1331 - accuracy: 0.9230 - val_loss: 10.1356 - val_accuracy: 0.0015
Epoch 5/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1297 - accuracy: 0.9275 - val_loss: 11.5640 - val_accuracy: 0.0044
Epoch 6/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1198 - accuracy: 0.9367 - val_loss: 14.6847 - val_accuracy: 0.0089
Epoch 7/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1112 - accuracy: 0.9442 - val_loss: 15.6482 - val_accuracy: 0.0162
Epoc

# total 데이터

## 데이터 전처리

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/서울시 빅데이터(2022-2)/논문/3. 폐업 여부 분류 모델/0) 데이터/data_total.csv')

In [ ]:
data['폐업여부']=data['폐업여부'].apply(lambda x: 1 if x=="Y" else 0)
data['프랜차이즈여부']=data['프랜차이즈여부'].apply(lambda x: 1 if x=="Y" else 0)

In [ ]:
data['표준산업분류코드']=data['표준산업분류코드'].astype('category')
data['행정동코드']=data['행정동코드'].astype('category')
data['분기']=data['분기'].astype('category')

In [ ]:
data_encoded = pd.get_dummies(data, columns=['표준산업분류코드', '행정동코드', '분기'])
data_encoded = data_encoded.drop(['점포명'], axis=1)

In [ ]:
train_data = data_encoded[data_encoded['년도'] == 2021]
test_data = data_encoded[data_encoded['년도'] == 2022]

In [ ]:
train_data = train_data.drop(['년도'], axis=1)
test_data = test_data.drop(['년도'], axis=1)

In [ ]:
x_train = train_data.drop(['폐업여부'], axis=1)
y_train = train_data['폐업여부']
x_test = test_data.drop(['폐업여부'], axis=1)
y_test = test_data['폐업여부']

In [ ]:
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

## 모델링

### 기본

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
614/614 [==============================] - 3s 4ms/step - loss: 0.1822 - accuracy: 0.9581 - val_loss: 0.0991 - val_accuracy: 0.9798
Epoch 2/100
614/614 [==============================] - 3s 4ms/step - loss: 0.1633 - accuracy: 0.9599 - val_loss: 0.1009 - val_accuracy: 0.9798
Epoch 3/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1613 - accuracy: 0.9599 - val_loss: 0.1000 - val_accuracy: 0.9798
Epoch 4/100
614/614 [==============================] - 2s 2ms/step - loss: 0.1598 - accuracy: 0.9599 - val_loss: 0.1033 - val_accuracy: 0.9798
Epoch 5/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1580 - accuracy: 0.9599 - val_loss: 0.1041 - val_accuracy: 0.9798
Epoch 6/100
614/614 [==============================] - 2s 4ms/step - loss: 0.1558 - accuracy: 0.9599 - val_loss: 0.1035 - val_accuracy: 0.9798
Epoch 7/100
614/614 [==============================] - 2s 2ms/step - loss: 0.1521 - accuracy: 0.9599 - val_loss: 0.1063 - val_accuracy: 0.9798

In [ ]:
y_pred = model.predict(x_test)

roc_score = roc_auc_score(y_test, y_pred)
print('ROC AUC 값: {0:.4f}'.format(roc_score))

precision, recall, _ = precision_recall_curve(y_test, y_pred)
pr_auc_score = auc(recall, precision)
print("PR AUC:", pr_auc_score)

725/725 [==============================] - 1s 1ms/step
ROC AUC 값: 0.5232
PR AUC: 0.041855889603964205


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
614/614 [==============================] - 3s 3ms/step - loss: 0.1896 - accuracy: 0.9522 - val_loss: 0.1010 - val_accuracy: 0.9798
Epoch 2/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1635 - accuracy: 0.9599 - val_loss: 0.0992 - val_accuracy: 0.9798
Epoch 3/100
614/614 [==============================] - 2s 4ms/step - loss: 0.1603 - accuracy: 0.9599 - val_loss: 0.1035 - val_accuracy: 0.9798
Epoch 4/100
614/614 [==============================] - 2s 4ms/step - loss: 0.1594 - accuracy: 0.9599 - val_loss: 0.1018 - val_accuracy: 0.9798
Epoch 5/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1564 - accuracy: 0.9599 - val_loss: 0.1052 - val_accuracy: 0.9798
Epoch 6/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1524 - accuracy: 0.9599 - val_loss: 0.1083 - val_accuracy: 0.9798
Epoch 7/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1473 - accuracy: 0.9599 - val_loss: 0.1169 - val_accuracy: 0.9798

In [ ]:
y_pred = model.predict(x_test)

roc_score = roc_auc_score(y_test, y_pred)
print('ROC AUC 값: {0:.4f}'.format(roc_score))

precision, recall, _ = precision_recall_curve(y_test, y_pred)
pr_auc_score = auc(recall, precision)
print("PR AUC:", pr_auc_score)

725/725 [==============================] - 1s 1ms/step
ROC AUC 값: 0.5372
PR AUC: 0.042358593485974275


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
614/614 [==============================] - 4s 5ms/step - loss: 0.1770 - accuracy: 0.9599 - val_loss: 0.1013 - val_accuracy: 0.9798
Epoch 2/100
614/614 [==============================] - 2s 4ms/step - loss: 0.1630 - accuracy: 0.9599 - val_loss: 0.0985 - val_accuracy: 0.9798
Epoch 3/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1602 - accuracy: 0.9599 - val_loss: 0.1019 - val_accuracy: 0.9798
Epoch 4/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1586 - accuracy: 0.9599 - val_loss: 0.1012 - val_accuracy: 0.9798
Epoch 5/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1549 - accuracy: 0.9599 - val_loss: 0.1053 - val_accuracy: 0.9798
Epoch 6/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1509 - accuracy: 0.9599 - val_loss: 0.1086 - val_accuracy: 0.9798
Epoch 7/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1450 - accuracy: 0.9599 - val_loss: 0.1143 - val_accuracy: 0.9798

In [ ]:
y_pred = model.predict(x_test)

roc_score = roc_auc_score(y_test, y_pred)
print('ROC AUC 값: {0:.4f}'.format(roc_score))

precision, recall, _ = precision_recall_curve(y_test, y_pred)
pr_auc_score = auc(recall, precision)
print("PR AUC:", pr_auc_score)

725/725 [==============================] - 1s 1ms/step
ROC AUC 값: 0.5317
PR AUC: 0.04146461261529248


### 앙상블

In [ ]:
roc_auc_scores = []
pr_auc_scores = []
final_y_train_pred_proba = []
final_y_test_pred_proba = []

num_iterations = 20

for iteration in range(num_iterations):

    non_closed_data = train_data[train_data['폐업여부'] == 0].sample(n=5000, random_state= iteration+426)
    closed_data = train_data[train_data['폐업여부'] == 1]

    new_train_data = pd.concat([non_closed_data, closed_data], axis=0)

    new_x_train = new_train_data.drop(['폐업여부'], axis=1)
    new_y_train = new_train_data['폐업여부']

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    new_x_train = tf.convert_to_tensor(new_x_train, dtype=tf.float32)
    new_y_train = tf.convert_to_tensor(new_y_train, dtype=tf.float32)

    model.fit(new_x_train, new_y_train, epochs=50, batch_size=64, validation_split=0.2)

    final_y_train_pred_proba.append(model.predict(x_train))
    final_y_test_pred_proba.append(model.predict(x_test))

final_y_train_pred_proba = np.mean(final_y_train_pred_proba, axis=0)
final_y_test_pred_proba = np.mean(final_y_test_pred_proba, axis=0)

final_roc_auc_test = roc_auc_score(y_train,final_y_train_pred_proba)
print("Final Train ROC AUC:", final_roc_auc_test)
precision, recall, _ = precision_recall_curve(y_train,final_y_train_pred_proba)
final_pr_auc_test = auc(recall, precision)
print("Final Train PR AUC:", final_pr_auc_test)

final_roc_auc_test = roc_auc_score(y_test, final_y_test_pred_proba)
print("Final Test ROC AUC:", final_roc_auc_test)
precision, recall, _ = precision_recall_curve(y_test, final_y_test_pred_proba)
final_pr_auc_test = auc(recall, precision)
print("Final Test PR AUC:", final_pr_auc_test)

Epoch 1/50
85/85 [==============================] - 1s 5ms/step - loss: 0.3485 - accuracy: 0.9036 - val_loss: 3.1050 - val_accuracy: 0.0000e+00
Epoch 2/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1800 - accuracy: 0.9230 - val_loss: 5.3781 - val_accuracy: 0.0000e+00
Epoch 3/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1484 - accuracy: 0.9230 - val_loss: 8.6556 - val_accuracy: 0.0000e+00
Epoch 4/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1378 - accuracy: 0.9230 - val_loss: 8.6251 - val_accuracy: 0.0000e+00
Epoch 5/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1307 - accuracy: 0.9265 - val_loss: 10.2303 - val_accuracy: 0.0170
Epoch 6/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1228 - accuracy: 0.9358 - val_loss: 10.9651 - val_accuracy: 0.0207
Epoch 7/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1210 - accuracy: 0.9383 - val_loss: 12.3514 - val_accuracy: 0.0089
E

# preindex 데이터

## 데이터 전처리

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/서울시 빅데이터(2022-2)/논문/3. 폐업 여부 분류 모델/0) 데이터/data_PredIndex.csv')

In [ ]:
data['폐업여부']=data['폐업여부'].apply(lambda x: 1 if x=="Y" else 0)
data['프랜차이즈여부']=data['프랜차이즈여부'].apply(lambda x: 1 if x=="Y" else 0)

In [ ]:
data['표준산업분류코드']=data['표준산업분류코드'].astype('category')
data['행정동코드']=data['행정동코드'].astype('category')
data['분기']=data['분기'].astype('category')

In [ ]:
data_encoded = pd.get_dummies(data, columns=['표준산업분류코드', '행정동코드', '분기'])
data_encoded = data_encoded.drop(['점포명'], axis=1)

In [ ]:
train_data = data_encoded[data_encoded['년도'] == 2021]
test_data = data_encoded[data_encoded['년도'] == 2022]

In [ ]:
train_data = train_data.drop(['년도'], axis=1)
test_data = test_data.drop(['년도'], axis=1)

In [ ]:
x_train = train_data.drop(['폐업여부'], axis=1)
y_train = train_data['폐업여부']
x_test = test_data.drop(['폐업여부'], axis=1)
y_test = test_data['폐업여부']

In [ ]:
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

## 모델링

### 기본

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
614/614 [==============================] - 4s 4ms/step - loss: 0.1900 - accuracy: 0.9516 - val_loss: 0.0988 - val_accuracy: 0.9798
Epoch 2/100
614/614 [==============================] - 4s 7ms/step - loss: 0.1618 - accuracy: 0.9599 - val_loss: 0.1003 - val_accuracy: 0.9798
Epoch 3/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1589 - accuracy: 0.9599 - val_loss: 0.1017 - val_accuracy: 0.9798
Epoch 4/100
614/614 [==============================] - 2s 2ms/step - loss: 0.1551 - accuracy: 0.9599 - val_loss: 0.1044 - val_accuracy: 0.9798
Epoch 5/100
614/614 [==============================] - 2s 2ms/step - loss: 0.1481 - accuracy: 0.9599 - val_loss: 0.1103 - val_accuracy: 0.9798
Epoch 6/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1385 - accuracy: 0.9599 - val_loss: 0.1169 - val_accuracy: 0.9798
Epoch 7/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1276 - accuracy: 0.9603 - val_loss: 0.1267 - val_accuracy: 0.9797

In [ ]:
y_pred = model.predict(x_test)

roc_score = roc_auc_score(y_test, y_pred)
print('ROC AUC 값: {0:.4f}'.format(roc_score))

precision, recall, _ = precision_recall_curve(y_test, y_pred)
pr_auc_score = auc(recall, precision)
print("PR AUC:", pr_auc_score)

725/725 [==============================] - 1s 2ms/step
ROC AUC 값: 0.5339
PR AUC: 0.04196926983340055


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
614/614 [==============================] - 3s 4ms/step - loss: 0.1874 - accuracy: 0.9572 - val_loss: 0.0995 - val_accuracy: 0.9798
Epoch 2/100
614/614 [==============================] - 3s 4ms/step - loss: 0.1625 - accuracy: 0.9599 - val_loss: 0.1003 - val_accuracy: 0.9798
Epoch 3/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1594 - accuracy: 0.9599 - val_loss: 0.1019 - val_accuracy: 0.9798
Epoch 4/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1542 - accuracy: 0.9599 - val_loss: 0.1049 - val_accuracy: 0.9798
Epoch 5/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1464 - accuracy: 0.9599 - val_loss: 0.1117 - val_accuracy: 0.9798
Epoch 6/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1364 - accuracy: 0.9599 - val_loss: 0.1264 - val_accuracy: 0.9798
Epoch 7/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1265 - accuracy: 0.9599 - val_loss: 0.1455 - val_accuracy: 0.9798

In [ ]:
y_pred = model.predict(x_test)

roc_score = roc_auc_score(y_test, y_pred)
print('ROC AUC 값: {0:.4f}'.format(roc_score))

precision, recall, _ = precision_recall_curve(y_test, y_pred)
pr_auc_score = auc(recall, precision)
print("PR AUC:", pr_auc_score)

725/725 [==============================] - 1s 2ms/step
ROC AUC 값: 0.5396
PR AUC: 0.04370896022392484


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
614/614 [==============================] - 4s 3ms/step - loss: 0.1822 - accuracy: 0.9599 - val_loss: 0.0986 - val_accuracy: 0.9798
Epoch 2/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1618 - accuracy: 0.9599 - val_loss: 0.1001 - val_accuracy: 0.9798
Epoch 3/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1595 - accuracy: 0.9599 - val_loss: 0.1025 - val_accuracy: 0.9798
Epoch 4/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1543 - accuracy: 0.9599 - val_loss: 0.1068 - val_accuracy: 0.9798
Epoch 5/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1464 - accuracy: 0.9599 - val_loss: 0.1172 - val_accuracy: 0.9798
Epoch 6/100
614/614 [==============================] - 2s 3ms/step - loss: 0.1354 - accuracy: 0.9600 - val_loss: 0.1340 - val_accuracy: 0.9797
Epoch 7/100
614/614 [==============================] - 3s 4ms/step - loss: 0.1228 - accuracy: 0.9607 - val_loss: 0.1529 - val_accuracy: 0.9762

In [ ]:
y_pred = model.predict(x_test)

roc_score = roc_auc_score(y_test, y_pred)
print('ROC AUC 값: {0:.4f}'.format(roc_score))

precision, recall, _ = precision_recall_curve(y_test, y_pred)
pr_auc_score = auc(recall, precision)
print("PR AUC:", pr_auc_score)

725/725 [==============================] - 1s 1ms/step
ROC AUC 값: 0.5433
PR AUC: 0.042946600255188996


### 앙상블

In [ ]:
roc_auc_scores = []
pr_auc_scores = []
final_y_train_pred_proba = []
final_y_test_pred_proba = []

num_iterations = 20

for iteration in range(num_iterations):

    non_closed_data = train_data[train_data['폐업여부'] == 0].sample(n=5000, random_state= iteration+426)
    closed_data = train_data[train_data['폐업여부'] == 1]

    new_train_data = pd.concat([non_closed_data, closed_data], axis=0)

    new_x_train = new_train_data.drop(['폐업여부'], axis=1)
    new_y_train = new_train_data['폐업여부']

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    new_x_train = tf.convert_to_tensor(new_x_train, dtype=tf.float32)
    new_y_train = tf.convert_to_tensor(new_y_train, dtype=tf.float32)

    model.fit(new_x_train, new_y_train, epochs=50, batch_size=64, validation_split=0.2)

    final_y_train_pred_proba.append(model.predict(x_train))
    final_y_test_pred_proba.append(model.predict(x_test))

final_y_train_pred_proba = np.mean(final_y_train_pred_proba, axis=0)
final_y_test_pred_proba = np.mean(final_y_test_pred_proba, axis=0)

final_roc_auc_test = roc_auc_score(y_train,final_y_train_pred_proba)
print("Final Train ROC AUC:", final_roc_auc_test)
precision, recall, _ = precision_recall_curve(y_train,final_y_train_pred_proba)
final_pr_auc_test = auc(recall, precision)
print("Final Train PR AUC:", final_pr_auc_test)

final_roc_auc_test = roc_auc_score(y_test, final_y_test_pred_proba)
print("Final Test ROC AUC:", final_roc_auc_test)
precision, recall, _ = precision_recall_curve(y_test, final_y_test_pred_proba)
final_pr_auc_test = auc(recall, precision)
print("Final Test PR AUC:", final_pr_auc_test)

Epoch 1/50
85/85 [==============================] - 2s 5ms/step - loss: 0.3125 - accuracy: 0.9184 - val_loss: 3.6821 - val_accuracy: 0.0000e+00
Epoch 2/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1638 - accuracy: 0.9230 - val_loss: 6.3871 - val_accuracy: 0.0000e+00
Epoch 3/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1398 - accuracy: 0.9230 - val_loss: 8.6693 - val_accuracy: 0.0059
Epoch 4/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1253 - accuracy: 0.9361 - val_loss: 10.4662 - val_accuracy: 0.0339
Epoch 5/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1145 - accuracy: 0.9424 - val_loss: 11.9822 - val_accuracy: 0.0303
Epoch 6/50
85/85 [==============================] - 0s 3ms/step - loss: 0.1040 - accuracy: 0.9489 - val_loss: 15.0775 - val_accuracy: 0.0295
Epoch 7/50
85/85 [==============================] - 0s 3ms/step - loss: 0.0936 - accuracy: 0.9544 - val_loss: 17.3316 - val_accuracy: 0.0391
Epoch 8/